In [ ]:
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
#!{sys.executable} -m pip install tensorflow-gpu==2.4
#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/bhawks/.conda/envs/dune-1dcnn/lib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
wireplane = "Z"
path = 'processed_data/'
filenames = [path+f for f in listdir(path) if (isfile(join(path, f)) and wireplane in f)]

In [ ]:
filenames

In [ ]:
for i in range(6):
    print(filenames[i])

In [ ]:
x_test = np.load(filenames[0])
x_train = np.load(filenames[1])
y_test_flat_noise_target = np.load(filenames[2])

y_test = np.load(filenames[3])
y_train_flat_noise_target = np.load(filenames[4])
y_train = np.load(filenames[5])

In [ ]:
num = 0
for idx in range(num,num+3):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 2))
    fig.suptitle('Waveforms')
    ax1.plot(x_train[idx],color='blue', label='clean')
    ax1.plot(y_train[idx],color='m',alpha=0.7, label='data')

    ax2.plot(y_train_flat_noise_target[idx],color='m',alpha=0.7, label='data')
    ax2.set_title("y_train_flat_noise_target")
    plt.show()

In [ ]:
mean = x_train.mean()
std = x_train.std()

In [ ]:
x_train_scaled = (x_train-mean)/std
x_test_scaled = (x_test-mean)/std
y_train_flat_noise_target_scaled = (y_train_flat_noise_target-mean)/std

y_train_scaled = (y_train-mean)/std
y_test_scaled = (y_test-mean)/std
y_test_flat_noise_target_scaled = (y_test_flat_noise_target-mean)/std



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, Dense,Flatten, AveragePooling1D
from tensorflow.keras.layers import Input,  UpSampling1D
from tensorflow.keras.models import Model
#from keras import backend as K

# Autoencoder

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("learn_models/" + wireplane + "plane_nu.h5")

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [ ]:
time_periods = 200

In [ ]:
input_wave = Input(shape=(x_train_scaled.shape[1], 1)) 
x = Conv1D(filters=16, kernel_size=3, strides=2, activation = "relu", weights=model.layers[0].get_weights() , input_shape=(time_periods,1))(input_wave)
x = MaxPooling1D(pool_size=2)(x)
 
#second convolutional block
x = Conv1D(filters=32, kernel_size=5, strides=2, weights=model.layers[2].get_weights(), activation = "relu", )(x)
x = MaxPooling1D(pool_size=2)(x)

encoded = Conv1D(filters=64, kernel_size=9, weights=model.layers[5].get_weights(), activation = "relu")(x)


x = Conv1D(filters=64, kernel_size=9, padding = "same", activation = "relu")(encoded)
x = UpSampling1D(4)(x)

x = Conv1D(filters=32, kernel_size=5, padding = "same", activation = "relu")(x)
x = UpSampling1D(3)(x)

x = Conv1D(filters=16, kernel_size=3,  activation = "relu")(x)
x = UpSampling1D(6)(x)

decoded = Conv1D(filters=1, kernel_size=5,  activation = "linear")(x)



autoencoder = Model(input_wave, decoded)



In [ ]:
for i,layer in enumerate(autoencoder.layers):                                      
  print(i,layer.name)


In [ ]:
for layer in autoencoder.layers[:6]:                                               
    layer.trainable=False                                                          
for layer in autoencoder.layers[6:]:                                               
    layer.trainable=True                                                           
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
autoencoder.summary()

### IMPORTANT: define the target for training
1) x_train_scale: train against itself (for wavefroms with signal + noise, target is that wavefrom), (for pure noise, the target will be that pure waveform as well)
2) y_train_flat_noise_target: same as (1), but when the wavefrom is JUST noise, target get will be ZERO
2) y_train_scale: train using clean signals as target

In [ ]:
#target = x_train_scaled
#target = y_train_flat_noise_target_scaled
target = y_train_scaled

tarID = 3

In [ ]:
%%time
history = autoencoder.fit(x_train_scaled,                                                              
                    target,                                                            
                    batch_size=1024,                                              
                    epochs=100,                                                      
                    callbacks=None, # callbacks=callbacks_list,                         
                    validation_split=0.2, shuffle=False,                                               
                    verbose=1)   

In [ ]:
plt.figure(figsize=(12, 8))                                                     
plt.plot(history.history['loss'], "r--", label="Loss of training data", antialiased=True)
plt.plot(history.history['val_loss'], "r", label="Loss of validation data", antialiased=True)
plt.title('Model Loss',fontsize=15)                                            
plt.ylabel('Loss (MSE)', fontsize=12)                                                 
plt.xlabel('Training Epoch', fontsize=12)                                                                                                                       
plt.legend(fontsize=12)                                                                    
plt.show()     

In [ ]:
%%time
y = autoencoder.predict(x_test_scaled, batch_size=1024)

In [ ]:
y = y.reshape(y.shape[0], y.shape[1])

In [ ]:
model.save("results/" + wireplane + "plane_nu_" + str(tarID) + ".h5")

In [ ]:
np.save('results/train_target_sig_' + wireplane + "_" + str(tarID), y)